1.图像转场动画

In [2]:
import cv2
import numpy as np
import imageio
import glob
import os
from PIL import Image, ImageDraw, ImageFont

In [13]:
# 调整图片大小函数
def resize_image(image, size):
    return cv2.resize(image, (size[1], size[0]))

# 创建过渡帧的函数
def create_transition_frames(image1, image2, transition_type, frames=30):
    h, w, _ = image1.shape
    transition_frames = []

    # 3个转场类型
    if transition_type == 'wipe_up':
        for i in range(frames):
            alpha = i / frames
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.rectangle(mask, (0, 0), (w, int(h * (1 - alpha))), 255, -1)  # 从上到下
            frame = cv2.bitwise_and(image1, image1, mask=cv2.bitwise_not(mask))
            frame += cv2.bitwise_and(image2, image2, mask=mask)
            transition_frames.append(frame)
    elif transition_type == 'wipe_down':
        for i in range(frames):
            alpha = i / frames
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.rectangle(mask, (0, int(h * alpha)), (w, h), 255, -1)  # 从下到上
            frame = cv2.bitwise_and(image1, image1, mask=cv2.bitwise_not(mask))
            frame += cv2.bitwise_and(image2, image2, mask=mask)
            transition_frames.append(frame)
    elif transition_type == 'rotate':
        for i in range(frames):
            alpha = i / frames
            M = cv2.getRotationMatrix2D((w / 2, h / 2), alpha * 360, 1)
            rotated_image1 = cv2.warpAffine(image1, M, (w, h))
            rotated_image2 = cv2.warpAffine(image2, M, (w, h))
            frame = cv2.addWeighted(rotated_image1, 1 - alpha, rotated_image2, alpha, 0)
            transition_frames.append(frame)

    return transition_frames

# 保存 GIF 动画
def save_gif(frames, save_path, fps=10):
    with imageio.get_writer(save_path, mode='I', fps=fps) as writer:
        for frame in frames:
            writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

def main():
    # 查找 JPG 文件
    image_files = sorted(glob.glob('*.jpg'))
    if len(image_files) < 2:
        raise ValueError('至少需要两张图片')

    image1 = cv2.imread(image_files[0])
    image2 = cv2.imread(image_files[1])
    
    # 确保两张图片大小一致
    if image1.shape != image2.shape:
        h, w, _ = min(image1.shape[0], image2.shape[0]), min(image1.shape[1], image2.shape[1]), 3
        image1 = resize_image(image1, (h, w))
        image2 = resize_image(image2, (h, w))

    transition_type = 'rotate'  # 还有 'wipe_down' 或 'rotate'
    frames = create_transition_frames(image1, image2, transition_type)

    save_gif(frames, 'transition_animation.gif')

if __name__ == '__main__':
    main()


2.滚动字幕

In [13]:
# 设置参数
image_path = 'input_image1.jpg'  
output_gif = 'scrolling_text.gif'  
text = '这是滚动的字幕，文字从右到左滚动显示。'  # 要滚动的文字
font_path = 'simsun.ttc'  # 字体路径
font_size = 30  # 字体大小
fps = 30  # 帧率
scroll_speed = 10  # 滚动速度（越大滚动越慢）
image_width = 400  # 输出图像宽度
image_height = 400  # 输出图像高度

# 读取背景图像
bg_image = cv2.imread(image_path)
bg_image = cv2.cvtColor(bg_image, cv2.COLOR_BGR2RGB)  # 确保图像颜色是 RGB
bg_image = cv2.resize(bg_image, (image_width, image_height))

# 使用Pillow创建滚动文本
def create_text_image(text, font_size):
    font = ImageFont.truetype(font_path, font_size)
    # 使用 getbbox 计算文本的边界框
    bbox = font.getbbox(text)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    image = Image.new('RGBA', (text_width, text_height), (255, 255, 255, 0))  # 使用 RGBA 
    draw = ImageDraw.Draw(image)
    draw.text((0, 0), text, font=font, fill=(0, 0, 0, 255))  # 添加 alpha 值
    return image

# 创建滚动字幕
text_image = create_text_image(text, font_size)

# GIF帧列表
frames = []

# 滚动字幕效果
for i in range(-text_image.width, image_width, scroll_speed):
    frame = bg_image.copy()
    frame_pil = Image.fromarray(frame)
    # 将背景图像转换为 'RGBA' 
    frame_pil = frame_pil.convert('RGBA')
    # 计算掩模
    mask = text_image.convert('L')
    # 粘贴文本图像
    frame_pil.paste(text_image, (image_width - text_image.width - i, image_height - text_image.height), mask=mask)
    frames.append(np.array(frame_pil))

# 保存为GIF
imageio.mimsave(output_gif, frames, fps=fps)

print(f"滚动字幕GIF已保存到 {output_gif}")


滚动字幕GIF已保存到 scrolling_text.gif


3.1前景目标在背景中移动

In [6]:
# 读取图像
image_path = 'input_image1.jpg'
image = cv2.imread(image_path)
height, width = image.shape[:2]

# 定义掩码和模型
mask = np.zeros((height, width), np.uint8)
bg_model = np.zeros((1, 65), np.float64)
fg_model = np.zeros((1, 65), np.float64)

# 3. 定义前景区域
# 手动选择矩形区域
r = cv2.selectROI("Select ROI", image, fromCenter=False, showCrosshair=True)

# 关闭窗口
cv2.destroyWindow("Select ROI")

# 获取矩形的坐标和尺寸
x, y, w, h = r
print(f"Selected region: x={x}, y={y}, width={w}, height={h}")

# 使用选择的矩形区域进行前景分割
rect = (x, y, x+w, y+h)

# 提取前景
cv2.grabCut(image, mask, rect, bg_model, fg_model, 5, cv2.GC_INIT_WITH_RECT)

# 将前景和背景分离
mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
foreground = image * mask2[:, :, np.newaxis]

# 创建动画效果
num_frames = 20
gif_images = []

for i in range(num_frames):
    shift = (i * 10) % width  # Move by 10 pixels per frame, adjust as needed
    
    # 将前景目标从背景中扣除
    background_without_foreground = image.copy()
    background_without_foreground[mask2 == 1] = [232, 163, 0]
    
    # 移动前景目标
    translation_matrix = np.float32([[1, 0, shift], [0, 1, 0]])
    shifted_foreground = cv2.warpAffine(foreground, translation_matrix, (width, height))
    
    # 合成背景和移动的前景
    result = np.where(shifted_foreground > 0, shifted_foreground, background_without_foreground)
    
    gif_images.append(Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB)))

# 保存为 GIF 动画
gif_path = 'animated_foreground.gif'
gif_images[0].save(gif_path, save_all=True, append_images=gif_images[1:], duration=100, loop=0)

print(f"GIF 动画已保存为: {gif_path}")


Selected region: x=1, y=11, width=363, height=463
GIF 动画已保存为: animated_foreground.gif


3.2前景目标局部运动的动画图像(挥手)

In [32]:

# 读取图像
image_path = 'input_image1.jpg'
image = cv2.imread(image_path)
height, width = image.shape[:2]

# 创建掩码和模型
mask = np.zeros((height, width), np.uint8)
bg_model = np.zeros((1, 65), np.float64)
fg_model = np.zeros((1, 65), np.float64)

# 定义前景区域
# 手动选择矩形区域
r = cv2.selectROI("Select ROI", image, fromCenter=False, showCrosshair=True)

# 关闭窗口
cv2.destroyWindow("Select ROI")

# 获取矩形的坐标和尺寸
x, y, w, h = r
print(f"Selected region: x={x}, y={y}, width={w}, height={h}")

# 前景分割
rect = (x, y, x+w, y+h)

# 提取前景
cv2.grabCut(image, mask, rect, bg_model, fg_model, 5, cv2.GC_INIT_WITH_RECT)

# 创建前景和背景图像
mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
foreground = image * mask2[:, :, np.newaxis]
background = image.copy()
background[mask2 == 1] = 0

# 定义旋转中心为矩形的右上角
rotation_center_x = x + w - 1
rotation_center_y = y

# 创建挥手动画效果
num_frames = 20
gif_images = []
rotation_angle_range = 30  # 最大旋转角度
rotation_step = 2  # 每帧旋转步长

# 手部区域在前景图中的位置
hand_x, hand_y, hand_w, hand_h = x, y, w, h

for i in range(num_frames):
    # 计算当前帧的旋转角度
    angle = rotation_angle_range * np.sin(i * 2 * np.pi / num_frames)
    
    # 创建旋转矩阵
    rotation_matrix = cv2.getRotationMatrix2D((rotation_center_x - hand_x, rotation_center_y - hand_y), angle, 1)
    
    # 执行旋转
    rotated_hand_area = cv2.warpAffine(foreground[hand_y:hand_y+hand_h, hand_x:hand_x+hand_w], 
                                       rotation_matrix, (hand_w, hand_h))
    
    # 将旋转后的手部区域合成到前景中
    rotated_foreground = foreground.copy()
    rotated_foreground[hand_y:hand_y+hand_h, hand_x:hand_x+hand_w] = rotated_hand_area
    
    # 合成背景和旋转后的前景
    result = background + rotated_foreground
    
   # 查找所有三个通道都是0的像素位置（即黑色像素）
    black_pixels = (result[:, :, 0] == 0) & (result[:, :, 1] == 0) & (result[:, :, 2] == 0)
    
    # 将这些像素的颜色改为深橙色 [232, 163, 0]
    result[black_pixels] = [232, 163, 0]
    
    # 转换为PIL图像并添加到GIF图像列表
    gif_images.append(Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB)))

# 保存为GIF动画
gif_path = 'hand_wave_animation.gif'
gif_images[0].save(gif_path, save_all=True, append_images=gif_images[1:], duration=100, loop=0)

print(f"GIF 动画已保存为: {gif_path}")


Selected region: x=22, y=393, width=56, height=71
GIF 动画已保存为: hand_wave_animation.gif
